In [45]:
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
import dotenv
from langchain_core.messages import HumanMessage

dotenv.load_dotenv()

True

In [46]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph = StateGraph(State)

In [47]:
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["yorkshire"]:
        return "It's cold and wet."
    else:
        return "It's warm and sunny."
tools = [get_weather]

In [48]:

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    
)


In [49]:
tools = [get_weather]

llm_with_tools = llm.bind_tools(tools)

tool_node = ToolNode(tools)

graph.add_node("tool_node", tool_node)

In [50]:
def prompt_node(state: State) -> State:
    new_message = llm_with_tools.invoke(state["messages"])
    return {"messages": [new_message]}

graph.add_node("prompt_node", prompt_node)
                                    
                                


In [51]:
def conditional_edge(state: State) -> Literal['tool_node', '__end__']:
    last_message = state["messages"][-1]
    if last_message.tool_calls:
        return "tool_node"
    else:
        return "__end__"
    
graph.add_conditional_edges(
    'prompt_node',
    conditional_edge
)
graph.add_edge("tool_node", "prompt_node")
graph.set_entry_point("prompt_node")

In [52]:
APP = graph.compile()

new_state = APP.invoke({"messages": ["Whats the weather in yorkshire?"]})

print(new_state["messages"][-1].content)
                    

The weather in Yorkshire is cold and wet.


In [ ]:
!pip install grandalf


   ---------------------------------------- 0/2 [pyparsing]
   -------------------- ------------------- 1/2 [grandalf]
   ---------------------------------------- 2/2 [grandalf]

Note: you may need to restart the kernel to use updated packages.


In [57]:
APP.get_graph().print_ascii()

          +-----------+             
          | __start__ |             
          +-----------+             
                 *                  
                 *                  
                 *                  
          +-------------+           
          | prompt_node |           
          +-------------+           
          ...         ***           
         .               *          
       ..                 **        
+---------+           +-----------+ 
| __end__ |           | tool_node | 
+---------+           +-----------+ 
